# Print Output Table From Bolocalc

This notebook, once pointed to an experiment directory, prints out a variety of relevant parameters (optical loading, NEPs, NETs) for each channel.  

In [99]:
import matplotlib.pyplot as plt
import numpy as np
import sys
#import os

In [100]:
# Import unpack.py from the BoloCalc/src/ directory
BoloCalcPath = 'BoloCalc/'
src_path = BoloCalcPath+'src'
if src_path not in sys.path:
    sys.path.append(src_path)

import unpack as up
# Instantiate an Unpack object
unpack = up.Unpack()   

# BoloCalc's "unpack" python interface does not help us access the Psats or
# safety factor, so we define it here by hand.  Change this if your simulation differs.
f_safety = 3.0

# Define which bolocalc simulation you're going to look at.
This means setting the site (Atacama or Pole) and band type (Real_Bands or Flat_Bands), which combine to tell us which directory to look at.

In [101]:
site = 'Pole'  # Pole or Atacama
band_type = 'Real_Bands_20210115'  #Flat_Bands or Real_Bands
#band_type = 'Flat_Bands'
#telescope_set = ['SAT_Bicep','SAT','TMALAT']  # SAT_500mK is another
telescope_set = ['SAT','TMALAT']

# band_type = 'Real_Bands'
# site = 'Atacama'
# telescope_set = ['SAT','CDLAT']  # use for Atacama

exp_dir = BoloCalcPath+'Experiments/'+site+'/'+band_type+'/'

In [102]:
# Load in the two dictionaries that contain all the info we need.
unpack.unpack_sensitivities(exp_dir)
unpack.unpack_optical_powers(exp_dir)
unpack.sens_outputs[band_type].keys()

dict_keys(['TMALAT', 'SAT', 'Summary'])

The next command shows that all the input Psat vectors are in any one of the camera files.  ie, MF and UHF input psat vectors are in the LF dictionary.

In [103]:
# Run this to see what's available in the sens_outputs
# unpack.sens_outputs[band_type]['SAT']['LF']['Summary']['LF_1']

# Print Optics information
Print out the relevant fields for each element in the optics chain:  power to detector, efficiency, and cumulative efficiency.

In [104]:
# A function that prints out one field (row) of a table, ie one optical element, 
# but all bands across the columns
def print_optics(site,band_type,telescope,field):
    firstch = 'LF_1'
    rows = unpack.pwr_outputs[band_type][telescope][tubes[0]]['Summary'][firstch].keys()
    for row in rows:
        list2print = [row.ljust(22)]
        for tube in tubes:
            if (site=='Pole' and telescope == 'TMALAT' and tube == 'LF'):
                chs = ['_1','_2','_3']
            else:
                chs = ['_1','_2']
            for ch in chs:
                chname = tube+ch
                pwrdict = unpack.pwr_outputs[band_type][telescope][tube]['Summary'][chname]
                list2print.append('{0:8.3f}'.format(pwrdict[row][field][0]))
        str2print = ''.join(list2print)
        print(str2print)

 
# Now let's use that function to print out tables for each telescope.
# 3 tables:  power to detector, efficiency, and cumulative efficiency.
for telescope in telescope_set:
    if (telescope == 'SAT' or telescope == 'SAT_500mK'):
        tubes = ['LF','MF1','MF2','UHF']
        col_list = ['30GHz ','40GHz ','85GHz ','145GHz','95GHz ','155GHz','220GHz','280GHz']
        col_str = 'Band:                    ' + '  '.join(col_list)
    else:
        tubes = ['LF','MF','UHF']
        if (site=='Pole' and telescope == 'TMALAT'):
            col_list = ['20GHz','30GHz ','40GHz ','90GHz ','150GHz','220GHz','280GHz']
        else:
            col_list = ['30GHz ','40GHz ','90GHz ','150GHz','220GHz','280GHz']
        col_str = 'Parameter:               ' + '  '.join(col_list)
                    
    ss = site + ', '+ band_type + ', ' + telescope

    for field in ['Power to Detector','Efficiency','Cumulative Efficiency']: 
        print(ss)
        print(field)
        print(col_str)
        print(site)
        print(band_type)
        print(telescope)
        print_optics(site,band_type,telescope,field)   
        print(" ")


Pole, Real_Bands_20210115, SAT
Power to Detector
Band:                    30GHz   40GHz   85GHz   145GHz  95GHz   155GHz  220GHz  280GHz
Pole
Real_Bands_20210115
SAT
CMB                      0.129   0.271   0.150   0.142   0.162   0.134   0.095   0.038
ATM                      0.223   1.733   1.929   1.894   1.714   1.970   5.703   7.414
Baffle                   0.168   0.412   0.374   0.678   0.443   0.724   1.290   1.160
UHMWWindow               0.085   0.310   0.624   1.931   0.824   2.205   5.792   6.563
IRShader1                0.004   0.013   0.025   0.079   0.033   0.090   0.236   0.265
IRShader2                0.003   0.011   0.021   0.066   0.028   0.076   0.197   0.222
IRShader3                0.003   0.009   0.019   0.058   0.025   0.067   0.174   0.195
IRShader4                0.002   0.008   0.015   0.047   0.020   0.054   0.140   0.157
IRShader5                0.002   0.006   0.013   0.039   0.017   0.045   0.116   0.130
IRShader6                0.001   0.005   0.010   0.

# Print Summary information

In [105]:
# Define a function that returns a string that will be the line in a table,
# where each line is something like Psat, NEP, etc, and each column is a
# different frequency band.

def getline (band_type,telescope,tubes,item):
    item_start = {'Psat':              'Psat:         ',
                  'Optical Throughput':'Optical Eff:  ',
                  'Total NEP':         'NEP_total:    ',
                  'Readout NEP':       'NEP_readout:  ',
                  'Optical Power':     'Opt. Power:   ', 
                  'Photon NEP':        'Phot. NEP:    ', 
                  'Bolometer NEP':     'Bolo NEP:     ',
                  'Detector NET_CMB':  'Det. NET_CMB: '}
    
    list2print = [item_start[item]]
    for tube in tubes:
        if (site=='Pole' and (telescope=='LAT' or telescope=='TMALAT') and tube=='LF'):
            chs=['_1','_2','_3']
        else:
            chs=['_1','_2']
        for ch in chs:
            chname = tube+ch
            data = unpack.sens_outputs[band_type][telescope][tube]['Summary'][chname]
            if item == 'Total NEP':
                NEP_photon = data['Photon NEP'][0]
                NEP_phonon = data['Bolometer NEP'][0]
                NEP_total = np.sqrt(NEP_photon**2+NEP_phonon**2)
                list2print.append('{0:8.2f}'.format(NEP_total))
            elif (item == 'Psat'):
                Psat = f_safety*data['Optical Power'][0]
                list2print.append('{0:8.2f}'.format(Psat))
            else:
                list2print.append('{0:8.2f}'.format(data[item][0]))
                
    str2print = ''.join(list2print)
    return str2print

# Now use that function to print out the tables.

# Select what rows to print out by putting them in this list.
rows = ['Optical Throughput','Optical Power','Psat','Photon NEP','Bolometer NEP','Readout NEP','Total NEP','Detector NET_CMB']

#Loop over all the telescopes.
for telescope in telescope_set:
    if (telescope == 'SAT' or telescope == 'SAT_500mK'):
        tubes = ['LF','MF1','MF2','UHF']
        col_list = ['30GHz ','40GHz ','85GHz ','145GHz','95GHz ','155GHz','220GHz','280GHz']
        col_str = 'Band:             ' + '  '.join(col_list)
    else:
        tubes = ['LF','MF','UHF']
        if (site=='Pole' and (telescope == 'LAT' or telescope=='TMALAT')):
            col_list = ['20GHz','30GHz ','40GHz ','90GHz ','150GHz','220GHz','280GHz']
        else:
            col_list = ['30GHz ','40GHz ','90GHz ','150GHz','220GHz','280GHz']
        col_str = 'Band              ' + '  '.join(col_list)
                    
    ss = site + ', '+ band_type + ', ' + telescope
    print(ss)
    print(col_str)

    for row in rows: 
        str2print = getline(band_type,telescope,tubes,row)   
        print(str2print)
    
    print(" ")
            

Pole, Real_Bands_20210115, SAT
Band:             30GHz   40GHz   85GHz   145GHz  95GHz   155GHz  220GHz  280GHz
Optical Eff:      0.52    0.61    0.46    0.55    0.49    0.55    0.56    0.51
Opt. Power:       0.68    2.82    3.31    5.08    3.39    5.53   14.17   16.61
Psat:             2.04    8.46    9.93   15.24   10.17   16.59   42.51   49.83
Phot. NEP:       11.05   32.59   38.18   50.93   37.94   54.18  104.64  125.13
Bolo NEP:         6.52   13.26   14.36   17.80   14.54   18.57   29.73   32.18
NEP_readout:      4.11   11.26   13.06   17.27   13.01   18.34   34.83   41.37
NEP_total:       12.83   35.18   40.79   53.95   40.63   57.27  108.78  129.20
Det. NET_CMB:   161.60  190.70  284.80  279.50  245.90  296.90  580.80 1394.10
 
Pole, Real_Bands_20210115, TMALAT
Band              20GHz  30GHz   40GHz   90GHz   150GHz  220GHz  280GHz
Optical Eff:      0.14    0.15    0.27    0.13    0.29    0.42    0.43
Opt. Power:       0.13    0.24    1.44    1.25    3.18    8.83   11.51
Psat: 

In [106]:
k = 1.38e-23
B = 0.27*90e9
Trj = 20.7
eta = 2.51e-12/(k*Trj*B)
eta

0.36159133609871474